In [1]:
import pandas as pd

# directory of example csv file
dir = "../data/Raw/example.csv"

# read and load csv file into a DataFrame without any headers
df = pd.read_csv(dir, header=None)

def convert_accel(df):
    """
        Function to:
            1. convert accelerometer data from raw bytes to g-force
            2. calculate the accuracy of the conversion with actual g-force values
    """

    # extract raw bytes data for x, y, z
    raw_bytes_x = df.iloc[:, [4, 5]]
    raw_bytes_y = df.iloc[:, [6, 7]]
    raw_bytes_z = df.iloc[:, [8, 9]]

    # extract actual g-force values for x, y, z
    g_force_x_answer = df.iloc[:, [16]]
    g_force_y_answer = df.iloc[:, [17]]
    g_force_z_answer = df.iloc[:, [18]]

    # calculate g-force for x, y, z using raw bytes
    g_force_x_cal = calculate_accel(raw_bytes_x)
    g_force_y_cal = calculate_accel(raw_bytes_y)
    g_force_z_cal = calculate_accel(raw_bytes_z)

    # calculate accuracy of conversion
    x_accuracy = get_accuracy(g_force_x_cal, g_force_x_answer)
    y_accuracy = get_accuracy(g_force_y_cal, g_force_y_answer)
    z_accuracy = get_accuracy(g_force_z_cal, g_force_z_answer)

    print("X Accuracy: ", x_accuracy)
    print("Y Accuracy: ", y_accuracy)
    print("Z Accuracy: ", z_accuracy)

def calculate_accel(raw_bytes):
    """
        Convert raw byte data of accelerometer to g-force
    """
    calculated_g_force = []

    for i in range(len(raw_bytes)):
        byte_high = raw_bytes.iloc[i, 0]
        byte_low = raw_bytes.iloc[i, 1]

        # combine high and low bytes
        combine = (byte_high << 8) | byte_low

        # convert 16-bit signed integer if negative
        if combine > 32767:
            combine -= 65536

        # define scale factor for converting to g-force
        scale_factor = 16 / 32768

        # convert to g-force
        g_force = combine * scale_factor

        # append to list
        calculated_g_force.append(g_force)

    # return in df
    return pd.DataFrame(calculated_g_force)

def get_accuracy(calculated, actual):
    """
        Calculate the accuracy of the conversion
        Compare values after rounding to 5 decimal places
    """
    count = 0
    for i in range(len(calculated)):
        # round to 5 decimal places and compare
        if round(calculated.iloc[i, 0], 5) == round(actual.iloc[i, 0], 5):
            count += 1

    return count / len(calculated)

# execute function
convert_accel(df)

X Accuracy:  1.0
Y Accuracy:  1.0
Z Accuracy:  1.0
